In [2]:
val creditRiskdf1 = spark.read.option("header","true").csv("creditRiskdf2_1.csv")
val creditRiskdf2 = spark.read.option("header","true").csv("creditRiskdf2_2.csv")
val creditRiskdf3 = spark.read.option("header","true").csv("creditRiskdf2_3.csv")

creditRiskdf1: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 186 more fields]
creditRiskdf2: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 186 more fields]
creditRiskdf3: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 186 more fields]


In [15]:
import org.apache.spark.sql
import org.apache.spark.sql._
import org.apache.spark.sql.types._
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.mllib.linalg.{Vector, Vectors}
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
import org.apache.spark.ml.feature.{VectorAssembler, StringIndexer}
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.Pipeline

import org.apache.spark.sql
import org.apache.spark.sql._
import org.apache.spark.sql.types._
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.mllib.linalg.{Vector, Vectors}
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
import org.apache.spark.ml.feature.{VectorAssembler, StringIndexer}
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.Pipeline


In [19]:
val creditRiskdf1and2 = creditRiskdf1.union(creditRiskdf2)
var creditRiskdf = creditRiskdf1and2.union(creditRiskdf3)

creditRiskdf1and2: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 186 more fields]
creditRiskdf: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 186 more fields]


In [5]:
var Array(train_df,test_df,validation_df)=creditRiskdf.randomSplit(
    Array(0.7,0.2,0.1))

train_df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 186 more fields]
test_df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 186 more fields]
validation_df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 186 more fields]


In [6]:
val creditRiskFeatures = creditRiskdf.columns
val features = creditRiskFeatures.filter(!_.contains("TARGET")).filter(!_.contains("SK_ID_CURR")) //Make sure we exclude the ID column and label column

creditRiskFeatures: Array[String] = Array(EXT_SOURCE_2, EXT_SOURCE_3, DAYS_BIRTH, AMT_CREDIT, AMT_ANNUITY, DAYS_ID_PUBLISH, previous_loans_CNT_PAYMENT_mean, previous_loans_SELLERPLACE_AREA_max, DAYS_LAST_PHONE_CHANGE, DAYS_REGISTRATION, CODE_GENDER_F, REGION_POPULATION_RELATIVE, client_cash_CNT_INSTALMENT_max_mean, bureau_DAYS_CREDIT_mean, previous_loans_NAME_CONTRACT_STATUS_Refused_count_norm, client_installments_NUM_INSTALMENT_VERSION_max_mean, bureau_DAYS_CREDIT_min, previous_loans_NAME_YIELD_GROUP_high_count_norm, previous_loans_RATE_DOWN_PAYMENT_sum, previous_loans_DAYS_FIRST_DUE_min, client_cash_SK_DPD_DEF_sum_max, bureau_AMT_CREDIT_SUM_LIMIT_mean, bureau_DAYS_CREDIT_sum, previous_loans_DAYS_LAST_DUE_1ST_VERSION_max, previous_loans_PRODUCT_COMBINATION_Cash X-Sell: low_count_norm, ...

In [20]:
val creditRiskFeatures = creditRiskdf.columns
for (colName<-creditRiskFeatures){
     |   creditRiskdf=creditRiskdf.withColumn(colName,col(colName).cast("Float"))
     | }

creditRiskFeatures: Array[String] = Array(EXT_SOURCE_2, EXT_SOURCE_3, DAYS_BIRTH, AMT_CREDIT, AMT_ANNUITY, DAYS_ID_PUBLISH, previous_loans_CNT_PAYMENT_mean, previous_loans_SELLERPLACE_AREA_max, DAYS_LAST_PHONE_CHANGE, DAYS_REGISTRATION, CODE_GENDER_F, REGION_POPULATION_RELATIVE, client_cash_CNT_INSTALMENT_max_mean, bureau_DAYS_CREDIT_mean, previous_loans_NAME_CONTRACT_STATUS_Refused_count_norm, client_installments_NUM_INSTALMENT_VERSION_max_mean, bureau_DAYS_CREDIT_min, previous_loans_NAME_YIELD_GROUP_high_count_norm, previous_loans_RATE_DOWN_PAYMENT_sum, previous_loans_DAYS_FIRST_DUE_min, client_cash_SK_DPD_DEF_sum_max, bureau_AMT_CREDIT_SUM_LIMIT_mean, bureau_DAYS_CREDIT_sum, previous_loans_DAYS_LAST_DUE_1ST_VERSION_max, previous_loans_PRODUCT_COMBINATION_Cash X-Sell: low_count_norm, ...

In [21]:
val df9=creditRiskdf.filter($"TARGET" === 1.0)
df9.count()

df9: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 186 more fields]
res8: Long = 2359


In [22]:
val df10=creditRiskdf.filter($"TARGET" === 0.0)
df10.count()

df10: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 186 more fields]
res9: Long = 33224


In [23]:
//Now from 33224 rows of TARGET==0, we randomly extract 2300 rows and combine it with df9
//To get a dataframe of 2300 target==1 and 2300 target ==0
var Array(df11,df12)=df10.randomSplit(
    Array(0.071,0.929))

df11: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 186 more fields]
df12: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 186 more fields]


In [24]:
val df13=df9.union(df11)

df13: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 186 more fields]


In [34]:
df13.count()//The df13 contains 2300 rows of TARGET==1 and 2300 rows of TARGET==0

res13: Long = 4704


In [52]:
df13.filter($"TARGET" === 0.0).count()


res24: Long = 2345


In [53]:
df13.filter($"TARGET" === 1.0).count()

res25: Long = 2359


In [25]:
val assembler = new VectorAssembler().setInputCols(features).setOutputCol("features")
val labelIndexer = new StringIndexer().setInputCol("TARGET").setOutputCol("label")
val model = new LogisticRegression()

assembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_7dfcbcd84946
labelIndexer: org.apache.spark.ml.feature.StringIndexer = strIdx_30afb35e5c2a
model: org.apache.spark.ml.classification.LogisticRegression = logreg_d87b5777f29e


In [36]:
val assembler = new VectorAssembler().setInputCols(features).setOutputCol("features")
val df2 = assembler.transform(df13)

assembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_32cc07b75833
df2: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 187 more fields]


In [37]:
val labelIndexer = new StringIndexer().setInputCol("TARGET").setOutputCol("label")
val df3 = labelIndexer.fit(df2).transform(df2)

labelIndexer: org.apache.spark.ml.feature.StringIndexer = strIdx_daa40bb8d872
df3: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 188 more fields]


In [38]:
val model = new LogisticRegression().fit(df3)
val predictions2 = model.transform(df3)

model: org.apache.spark.ml.classification.LogisticRegressionModel = LogisticRegressionModel: uid = logreg_26f22abc20b5, numClasses = 2, numFeatures = 186
predictions2: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 191 more fields]


In [41]:
import org.apache.spark.rdd.RDD
import org.apache.spark.mllib.evaluation.MulticlassMetrics
import org.apache.spark.mllib.regression.LabeledPoint
import org.apache.spark.mllib.util.MLUtils
import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics

import org.apache.spark.rdd.RDD
import org.apache.spark.mllib.evaluation.MulticlassMetrics
import org.apache.spark.mllib.regression.LabeledPoint
import org.apache.spark.mllib.util.MLUtils
import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics


In [39]:
val predictions2 = lrModel.transform(df13)
val predictionLabelsRDD1 = predictions2.select("prediction", "label").rdd.map(r => (r.getDouble(0), r.getDouble(1)))
val binMetrics1 = new BinaryClassificationMetrics(predictionLabelsRDD1)
println("area under the receiver operating characteristic (ROC) curve : " + binMetrics1.areaUnderROC)

area under the receiver operating characteristic (ROC) curve : 0.7051602943316483


predictions2: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 191 more fields]
predictionLabelsRDD1: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[1756] at map at <console>:53
binMetrics1: org.apache.spark.mllib.evaluation.BinaryClassificationMetrics = org.apache.spark.mllib.evaluation.BinaryClassificationMetrics@4206f5e1


In [43]:
val lp = predictions2.select( "label", "prediction")
val counttotal = predictions2.count()
val correct = lp.filter($"label" === $"prediction").count() //correct prediction
val wrong = lp.filter(not($"label" === $"prediction")).count() //wrong prediction
val truep = lp.filter($"prediction" === 0.0).filter($"label" === $"prediction").count() //Correct prediction of label=0
val truen = lp.filter($"prediction" === 1.0).filter($"label" === $"prediction").count() //Correct prediction of label=1
val falseN = lp.filter($"prediction" === 0.0).filter(not($"label" === $"prediction")).count() //Wrong prediction when label=1
val falseP = lp.filter($"prediction" === 1.0).filter(not($"label" === $"prediction")).count() //Wrong prediction when lable =0
val ratioWrong=wrong.toDouble/counttotal.toDouble //Wrong prediction ratio
val ratioCorrect=correct.toDouble/counttotal.toDouble //Correct predicition ratio

lp: org.apache.spark.sql.DataFrame = [label: double, prediction: double]
counttotal: Long = 4704
correct: Long = 3317
wrong: Long = 1387
truep: Long = 1651
truen: Long = 1666
falseN: Long = 679
falseP: Long = 708
ratioWrong: Double = 0.29485544217687076
ratioCorrect: Double = 0.7051445578231292


In [35]:
predictions2.select ("SK_ID_CURR","features", "label","probability", "prediction").show()

+----------+--------------------+-----+--------------------+----------+
|SK_ID_CURR|            features|label|         probability|prediction|
+----------+--------------------+-----+--------------------+----------+
|  100049.0|(186,[0,1,2,3,4,5...|  0.0|[0.41297183801247...|       1.0|
|  100902.0|(186,[0,1,2,3,4,5...|  0.0|[0.40944020354736...|       1.0|
|  101042.0|(186,[0,1,2,3,4,5...|  0.0|[0.38802373681337...|       1.0|
|  101197.0|(186,[0,1,2,3,4,5...|  0.0|[0.63103137138405...|       0.0|
|  101277.0|(186,[0,1,2,3,4,5...|  0.0|[0.91460324467911...|       0.0|
|  101286.0|(186,[0,1,2,3,4,5...|  0.0|[0.60208368726355...|       0.0|
|  101495.0|(186,[0,1,2,3,4,5...|  0.0|[0.51719934824546...|       0.0|
|  101859.0|(186,[0,1,2,3,4,5...|  0.0|[0.63584047678167...|       0.0|
|  101980.0|(186,[0,1,2,3,4,5...|  0.0|[0.72854178873096...|       0.0|
|  102003.0|(186,[0,1,2,3,4,5...|  0.0|[0.78686212416355...|       0.0|
|  102025.0|(186,[0,1,2,3,4,5...|  0.0|[0.67750359233151...|    